In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet50
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms.functional as F
import seaborn as sns
from tqdm import tqdm
import time
import pickle as pkl

In [ ]:
# print(torch.cuda.is_available())
# print(torch.cuda.current_device())
# print(torch.cuda.get_device_name(torch.cuda.current_device()))
# print(torch.cuda.memory_summary(device=None, abbreviated=False))

device = torch.device(torch.cuda.current_device() if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Load dataset
dataset_dir = "dataset"

In [ ]:
# Get image size from the first image
first_image_file = os.listdir(f"{dataset_dir}/Non Demented")[0]
img = plt.imread(f"{dataset_dir}/Non Demented/{first_image_file}")

img_height, img_width, _ = img.shape

print(f"Image size: {img_height}x{img_width}")

In [ ]:
# Set batch size
batch_size = 16

In [ ]:
# Define transformations
data_transforms = transforms.Compose([
    # transforms.Resize((250, 250)),
    transforms.RandomResizedCrop(img_height),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the datasets
dataset = ImageFolder(dataset_dir, transform=data_transforms)

# Create train and test data splits
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Number of classes
num_classes = len(dataset.classes)
num_classes

In [ ]:
# Set number of epochs
epochs_number = 20

#### Testing pretrained ResNet-50 model


In [ ]:
# Load the ResNet50 model
base_resNet_model = resnet50()

# Freeze convolutional layers
for param in base_resNet_model.parameters():
    param.requires_grad = False

# Add custom classification head
num_ftrs = base_resNet_model.fc.in_features
base_resNet_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes),
    nn.LogSoftmax(dim=1)
)

# Define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(base_resNet_model.fc.parameters(), lr=0.001)

In [ ]:
# Store accuracy after each epoch
train_accuracy = []
val_accuracy = []

# Move model to device
base_resNet_model = base_resNet_model.to(device)

# Train model
for epoch in tqdm(range(epochs_number), desc="Epochs"):
    # start_time = time.time()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for i, (inputs, labels) in enumerate(train_loader, start=1):
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = base_resNet_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Print file number
        print(f"\r[{i}/{len(train_loader)}]", end="")

    # Compute accuracy
    train_acc = correct / total
    train_accuracy.append(train_acc)

    # Validation loop
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = base_resNet_model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Compute accuracy
    val_acc = correct / total
    val_accuracy.append(val_acc)

    # Print statistics
    print(f"Train loss: {running_loss/len(train_loader):.3f} | Train accuracy: {train_acc:.3f} | Validation accuracy: {val_acc:.3f}")

In [ ]:
# Save trained model
torch.save(base_resNet_model.state_dict(), "resnet.pth")

In [ ]:
# Plot training and validation accuracy
plt.plot(train_accuracy, label="Training accuracy")
plt.plot(val_accuracy, label="Validation accuracy")
plt.title("Training and validation accuracy - ResNet50V2")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# # Load model
# base_resNet_model.load_state_dict(torch.load("resnet.pth"))

#### Testing DenseNet-121 model


In [ ]:
# # Load pre-trained DenseNet121 model
# base_denseNet_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# # Add custom classification head
# x = base_denseNet_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024)(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(num_classes, activation="softmax")(x)

# # Create model
# denseNet_model = Model(inputs=base_denseNet_model.input, outputs=predictions)

# # Freeze convolutional layers
# for layer in base_denseNet_model.layers:
#     layer.trainable = False

# # Compile model
# optimizer = Adam(learning_rate=0.001)
# denseNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# # denseNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# # Train model
# denseNet_history = denseNet_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# # Save trained model
# denseNet_model.save("denseNet_model.h5")

In [ ]:
# plt.plot(denseNet_history.history["accuracy"], label="Training accuracy")
# plt.plot(denseNet_history.history["val_accuracy"], label="Validation accuracy")
# plt.title("Training and validation accuracy - DenseNet121")
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.show()

#### Testing EfficientNetB7 model


In [ ]:
# # Load the ResNet50 model
# base_efficientNet_model = EfficientNetB7(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# # Add custom classification head
# x = base_efficientNet_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024)(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(num_classes, activation="softmax")(x)

# # Create model
# efficientNet_model = Model(inputs=base_efficientNet_model.input, outputs=predictions)

# # Freeze convolutional layers
# for layer in base_efficientNet_model.layers:
#     layer.trainable = False

# # Compile model
# optimizer = Adam(learning_rate=0.001)
# efficientNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# # efficientNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# # Train the model
# efficientNet_history = efficientNet_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# # Save the trained model
# efficientNet_model.save("efficientNet_model.h5")

### Testing custom CNN model


In [ ]:
# # Preprocess data
# data_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

# dataset = ImageFolder(dataset_dir, transform=data_transforms)

# # Define the train-test split
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size

# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# data_iter = iter(train_loader)
# images, labels = next(data_iter)

In [ ]:
# mean = np.array([0.485, 0.456, 0.406])
# std = np.array([0.229, 0.224, 0.225])
# images = (images.numpy().transpose((0, 2, 3, 1)) * std + mean).clip(0, 1)

In [ ]:
# class TumorClassifier(nn.Module):
#     def __init__(self, num_classes):
#         super(TumorClassifier, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(16, 32, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.classifier = nn.Sequential(
#             nn.Linear(32 * 56 * 56, 128),
#             nn.ReLU(inplace=True),
#             nn.Linear(128, num_classes)
#         )

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         return x

# model = TumorClassifier(num_classes=num_classes)
# # Load a pre-trained ResNet model and modify the classifier
# model.to(device)

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# # Initialize lists to store training history
# train_losses = []
# val_losses = []
# train_accuracies = []
# val_accuracies = []

In [ ]:
# # Training loop
# num_epochs = 20
# best_val_accuracy = 0.0

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0.0
#     correct = 0
#     total = 0
    
#     for batch_idx, (inputs, labels) in enumerate(train_loader):
#         inputs, labels = inputs.to(device), labels.to(device)
        
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         train_loss += loss.item()
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
    
#     train_accuracy = correct / total
#     train_losses.append(train_loss)
#     train_accuracies.append(train_accuracy)
    
#     # Validation
#     model.eval()
#     val_loss = 0.0
#     correct = 0
#     total = 0
    
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
            
#             val_loss += loss.item()
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
            
#     val_loss /= len(val_loader)
#     val_accuracy = correct / total
#     val_losses.append(val_loss)
#     val_accuracies.append(val_accuracy)
    
#     print(f'Epoch [{epoch+1}/{num_epochs}], '
#           f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, '
#           f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

    
#     # Save the best model
#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         torch.save(model.state_dict(), 'best_model.pth')

#### Tensorflow Version

In [ ]:
# # Custom CNN architecture
# custom_model = Sequential(
#     [
#         Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation="relu"),
#         MaxPooling2D((2, 2)),
#         Conv2D(128, (3, 3), activation="relu"),
#         MaxPooling2D((2, 2)),
#         Conv2D(128, (3, 3), activation="relu"),
#         MaxPooling2D((2, 2)),
#         Flatten(),
#         Dense(512, activation="relu"),
#         Dropout(0.5),
#         Dense(num_classes, activation="softmax"),
#     ]
# )

# # Compile the model
# optimizer = Adam(learning_rate=0.001)
# custom_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# # custom_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# # Train the model
# custom_CNN_history = custom_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# # Save the trained model
# custom_model.save("custom_CNN_model.h5")

In [ ]:
# plt.plot(custom_CNN_history.history["accuracy"], label="Training accuracy")
# plt.plot(custom_CNN_history.history["val_accuracy"], label="Validation accuracy")
# plt.title("Training and validation accuracy - Custom CNN")
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.show()

#### Evaluate models


In [ ]:
# # Table comparing the performance of models
# resNet_accuracy = resNet_history.history["accuracy"][-1]
# resNet_val_accuracy = resNet_history.history["val_accuracy"][-1]
# denseNet_accuracy = denseNet_history.history["accuracy"][-1]
# denseNet_val_accuracy = denseNet_history.history["val_accuracy"][-1]
# custom_CNN_accuracy = custom_CNN_history.history["accuracy"][-1]
# custom_CNN_val_accuracy = custom_CNN_history.history["val_accuracy"][-1]
# efficientNet_accuracy = efficientNet_history.history["accuracy"][-1]
# efficientNet_val_accuracy = efficientNet_history.history["val_accuracy"][-1]

# # resNet_f1 = resNet_history.history["f1_m"][-1]
# # resNet_val_f1 = resNet_history.history["val_f1_m"][-1]
# # denseNet_f1 = denseNet_history.history["f1_m"][-1]
# # denseNet_val_f1 = denseNet_history.history["val_f1_m"][-1]
# # custom_CNN_f1 = custom_CNN_history.history["f1_m"][-1]
# # custom_CNN_val_f1 = custom_CNN_history.history["val_f1_m"][-1]
# # efficientNet_f1 = efficientNet_history.history["f1_m"][-1]
# # efficientNet_val_f1 = efficientNet_history.history["val_f1_m"][-1]

# model_comparison = pd.DataFrame(
#     {
#         "Model": ["ResNet50V2", "DenseNet121", "Custom CNN", "EfficientNetB7"],
#         "Train Accuracy": [resNet_accuracy, denseNet_accuracy, custom_CNN_accuracy, efficientNet_accuracy],
#         "Validation Accuracy": [resNet_val_accuracy, denseNet_val_accuracy, custom_CNN_val_accuracy, efficientNet_val_accuracy],
#         # "Training F1 Score": [resNet_f1, denseNet_f1, custom_CNN_f1, efficientNet_f1],
#         # "Validation F1 Score": [resNet_val_f1, denseNet_val_f1, custom_CNN_val_f1, efficientNet_val_f1],
#     }
# )

# model_comparison